In [ ]:
!pip install pydicom
!conda install plotly
!conda install scikit-image
!python -m pip install pillow
!pip install python-gdcm
!conda install -c conda-forge gdcm
!pip install pylibjpeg pylibjpeg-libjpeg

from pydicom import dcmread
import pylibjpeg

import numpy as np
import pydicom
import pydicom.pixel_data_handlers.gdcm_handler as gdcm_handler
pydicom.config.image_handlers = [None, gdcm_handler]
pydicom.config.image_handlers = ['gdcm_handler']
import os
import matplotlib.pyplot as plt
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage import exposure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot #,iplot
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *
from pydicom.data import get_testdata_files

#init_notebook_mode(connected=True)
#print(pydicom.__version__)


/bin/bash: conda: command not found
/bin/bash: conda: command not found
/bin/bash: conda: command not found
     |████████████████████████████████| 4.2 MB 6.6 MB/s 
     |████████████████████████████████| 970 kB 54.1 MB/s 
     |████████████████████████████████| 3.6 MB 51.9 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = "/content/drive/My Drive/Workspace/DentAssistant/dataset/"
output_path = working_path = "/content/drive/My Drive/Workspace/DentAssistant/"

g = glob(data_path + '/*.dcm')

# Print out the first 5 file names to verify we're in the right folder.
print ("Total of %d DICOM images.\nFirst 5 filenames:" % len(g))
print ('\n'.join(g[:5]))
output_path = working_path = "/Users/narges/Workspace/DentAssistant/"

Total of 458 DICOM images.
First 5 filenames:
/content/drive/My Drive/Workspace/DentAssistant/dataset/I0000470295 (2).dcm
/content/drive/My Drive/Workspace/DentAssistant/dataset/I0000470289.dcm
/content/drive/My Drive/Workspace/DentAssistant/dataset/I0000470297.dcm
/content/drive/My Drive/Workspace/DentAssistant/dataset/I0000470295 (1).dcm
/content/drive/My Drive/Workspace/DentAssistant/dataset/I0000470291.dcm


In [ ]:
#
# Loop over the image files and store everything into a list.
#

def load_scan(path):
    slices = []
    # count = 0
    for s in os.listdir(path):
        ds = pydicom.dcmread(path + '/' + s, force=True)
        ds.PhotometricInterpretation = 'YBR_FULL'

        if s != '.DS_Store': #This is for AttributeError: 'FileDataset' object has no attribute 'InstanceNumber'
            # count += 1
            # print(count)
            # print(s)
            #print(ds)
            # print(int(ds.InstanceNumber), '\n')
            slices.append(ds)

    #slices = [pydicom.read_file(path + '/' + s, force=True) for s in os.listdir(path)]
    slices.sort(key=lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)

    for s in slices:
        s.SliceThickness = slice_thickness

    return slices


def get_pixels_hu(scans):
    # pix = [s.pixel_array for s in scans]
    # ds.PixelData = pix.tobytes()
    # print(scans[1])

    # img_list is of type generator numpy.ndarray
    #for s in range(len(scans)):
    #    print(scans[s].pixel_array)
    #    img_list = scans[s].pixel_array
    # print(type(img_list))
    #scans = (t.decompress() for t in scans)
    #print(scans)
    image = np.stack([s.pixel_array for s in scans])
    # print(np.array(image))

    # Convert to int16 (from sometimes int16),
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 1
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0

    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope

    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)

    image += np.int16(intercept)

    return np.array(image, dtype=np.int16)


def imageCount(dirName):
    """Counts the number of files in a directory
    Counts the number of files in a given directory and returns the result. Ignores desktop.ini to allow for Windows compatibility
    Args:
        dirName: A string representation of the name of the directory to be opened. This directory should be a child of the working directory. EX: "trainingImages"
    Returns:
        An integer value for the number of files in the directory
    """

    for root, dir, files in os.walk(dirName):
        scans = [file  for file in files if file != "desktop.ini"]
        return len(scans)
    print(dirName + " is not a valid directory")
    exit(-1)


def readScan(scanNum, dirName):
    """ Reads a DICOM file from a directory
    Given a directory name and the number of the file containing the image to be read, reads in and returns a numpy representation of the image. Also normalizes the pixel values of the image.
    Args:
        scanNum: An integer containing the one-based index of the file to be read
        dirName: A string representation of the name of the directory to be opened. This directory should be a child of the working directory. Ex: "trainingImages"
    Returns:
        A numpy array containing the pixel data for the image
    """

    for root, dir, files in os.walk(dirName):
        scans = [file  for file in files if file != "desktop.ini" or '.DS_Store']
        print("Reading image " + scans[scanNum] + " from " + dirName)
        data = np.load(dirName + scans[scanNum])
        #plt.imshow(data)
        #plt.show()
        data = exposure.equalize_adapthist(data)
        #plt.imshow(data)
        #plt.show()
        return data
    print(dirName + " is not a valid directory")
    exit(-1)


def readValidationImages(dirName):
    """ Reads in all images in the validation directories
    Creates a numpy array containing the pixel data of all the images in the directories of validation images. Assumes that the subdirectories "PosVal" and "NegVal" exit in the working directory

    Args: None
    Returns:
        A numpy array containing the pixel values for all validation images. Also returns a numpy array containing whether each image has a positive or negative label
    """

    labels = []
    patientData = []

    for i in range(0, imageCount(dirName)):
        data = readScan(i, dirName)
        if data != '.DS_Store':
            labels.append(1)
            patientData.append(data)
    """for i in range(0, imageCount("NegVal")):
        data = readScan(i, "NegVal")
        labels.append(0)
        patientData.append(data)"""

    patientData = np.stack(patientData).astype(float)

    return patientData#, np.stack(labels)


id = 0
patient = load_scan(data_path)
#print(patient)
imgs = get_pixels_hu(patient)
#imgs = readValidationImages(data_path)
print(imgs)


In [ ]:
np.save(output_path + "fullimages_%d.npy" % (id), imgs)

In [ ]:
file_used=output_path+"fullimages_%d.npy" % id
imgs_to_process = np.load(file_used).astype(np.float64)

plt.hist(imgs_to_process.flatten(), bins=50, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()


In [ ]:
id = 0
imgs_to_process = np.load(output_path+'fullimages_{}.npy'.format(id))

def sample_stack(stack, rows=6, cols=6, start_with=0, show_every=0):
    fig,ax = plt.subplots(rows,cols,figsize=[12,12])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title('slice %d' % ind)
        ax[int(i/rows),int(i % rows)].imshow(stack[ind],cmap='gray')
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()

sample_stack(imgs_to_process)

In [ ]:
print("Slice Thickness: %f" % patient[0].SliceThickness)
print("Pixel Spacing (row, col): (%f, %f) " % (patient[0].PixelSpacing[0], patient[0].PixelSpacing[1]))


In [ ]:
def make_mesh(image, threshold=-300, step_size=1):
    print("Transposing surface")
    p = image.transpose(2, 1, 0)

    print("Calculating surface")
    verts, faces, norm, val = measure.marching_cubes(p, threshold, step_size=step_size, allow_degenerate=True)
    return verts, faces


def plotly_3d(verts, faces):
    x, y, z = zip(*verts)

    print("Drawing")

    # Make the colormap single color since the axes are positional not intensity.
    #    colormap=['rgb(255,105,180)','rgb(255,255,51)','rgb(0,191,255)']
    colormap = ['rgb(236, 236, 212)', 'rgb(236, 236, 212)']

    fig = FF.create_trisurf(x=x,
                            y=y,
                            z=z,
                            plot_edges=False,
                            colormap=colormap,
                            simplices=faces,
                            backgroundcolor='rgb(64, 64, 64)',
                            title="Interactive Visualization")
    iplot(fig)


def plt_3d(verts, faces):
    print("Drawing")
    x, y, z = zip(*verts)
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], linewidths=0.05, alpha=1)
    face_color = [1, 1, 0.9]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, max(x))
    ax.set_ylim(0, max(y))
    ax.set_zlim(0, max(z))
    ax.set_axis_bgcolor((0.7, 0.7, 0.7))
    plt.show()

    """v, f = make_mesh(imgs_after_resamp, 350)
    plt_3d(v, f)


##########################################################################################################

v, f = make_mesh(imgs_after_resamp, 350, 2)
plotly_3d(v, f)"""

In [ ]:

"""id = 0
imgs_to_process = np.load(output_path + 'fullimages_{}.npy'.format(id))


def resample(image, scan, new_spacing=[1, 1, 1]):
    # Determine current pixel spacing
    spacing = ([float(scan[0].SliceThickness)] + [float(scan[0].PixelSpacing)])
    spacing = np.array(spacing)

    resize_factor = [spacing[0] / new_spacing[0], spacing[1] / new_spacing[1], spacing[2] / new_spacing[2]]
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor

    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)

    return image, new_spacing


print("Shape before resampling\t", imgs_to_process.shape)
imgs_after_resamp, spacing = resample(imgs_to_process, patient, [1, 1, 1])
print("Shape after resampling\t", imgs_after_resamp.shape)"""
